## Clean data files and save them on disk

In [ ]:
def clean_data(old_file_address, new_file_address):
    
    import string
    from unicodedata import normalize
    import re
    
    remove_punct_map = dict.fromkeys(map(ord, string.punctuation)) # thank you Reed!
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    
    with open(old_file_address, 'r') as old_file, open(new_file_address, 'w') as clean_file:
        for line in old_file:
            
            # normalize all Unicode characters to ASCII (maybe not nesessary)
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            
            # remove non-printable characters
            line = re_print.sub('', line)
            
            # normalize to lowercase, remove punctuation and write to a file
            clean_file.write(line.lower().translate(remove_punct_map))


In [ ]:
clean_data('europarl-v7esp.txt', 'clean_spanish.txt')

In [ ]:
clean_data('europarl-v7eng.txt', 'clean_english.txt')

## Data Observation with MRJob

In [ ]:
# Data contains millions of rows, so I used MRJob to handle it
# Use boilerplate to count the number of words and lines

from mrjob.job import MRJob
import string

remove_punct_map = dict.fromkeys(map(ord, string.punctuation)) #thank you Reed

class TextOverview(MRJob):

    def mapper(self, _, line):
        
        line = line.translate(remove_punct_map)
        line = line.lower()
        
        yield "total_words", len(line.split())
        yield "lines", 1

    def reducer(self, key, values):
        yield key, sum(values)


if __name__ == '__main__':
    TextOverview.run()
    
#python engl-span.py clean_english.txt
#python engl-span.py clean_spanish.txt

In [ ]:
# Count a number of unique words in each dataset:

from mrjob.job import MRJob
import string

remove_punct_map = dict.fromkeys(map(ord, string.punctuation))

class TextOverview(MRJob):

    def mapper(self, _, line):

        line = line.translate(remove_punct_map)
        line = line.lower()
        for word in line.split():
            yield 'words', word

    def reducer(self, key, values):
        yield key, len(set(values))
    

if __name__ == '__main__':
    TextOverview.run()

#python engl-span.py clean_english.txt
#python engl-span.py clean_spanish.txt

***Output for english file:***
- "total_words"   48 978 039
- "lines" 1 965 734
- "unique_words" 133 052

***Output for spanish file:***
- "total_words"   51 505 465
- "lines" 1 965 734
- "unique_words" 192 038

## Prepare datafiles

In [ ]:
# save combined clean txt file on disk just in case
with open('clean_english.txt', 'r') as eng, open('clean_spanish.txt', 'r') as span, open('combined.txt','w') as comb:
    
        for eline, sline in zip(eng, span):
            eline = eline.rstrip()
            sline = sline.rstrip()
            
            comb.write(f'{eline}\t{sline}\n')


In [ ]:
def to_pairs(doc):
    file = open(doc, mode='r', encoding='utf-8')
    text = file.read()
    file.close()
    lines = text.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

In [ ]:
# merge both files in one
pairs = to_pairs('combined.txt')

In [ ]:
pairs[0]

In [ ]:
def create_array(eng_file_address, esp_file_address):
    from numpy import array
    
    all_pairs = []
    with open(eng_file_address, 'r') as eng, open(esp_file_address, 'r') as span:
    
        for eline, sline in zip(eng, span):
            eline = eline.rstrip()
            sline = sline.rstrip()

            all_pairs.append([eline, sline])
        
    return array(all_pairs)

In [ ]:
pairs_array = create_array('clean_english.txt', 'clean_spanish.txt')

In [ ]:
for x in range(0,5):
    print(f'{x}. {pairs_array[x,0]} : {pairs_array[x,1]}')

## Save data to plk file

In [ ]:
def save_data(pairs, filename):

    from sklearn.externals import joblib
    
    joblib.dump(pairs, filename) 
    
    print(f'Saved: {filename}')

In [ ]:
save_data(pairs_array, 'english-spanish.pkl') #64Gb wow!!

## Split dataset

In [88]:
from sklearn.externals import joblib
import numpy as np

In [5]:
file = joblib.load('english-spanish.pkl')

In [6]:
shuffle(file)

In [30]:
dataset = file[:20000, :]

In [31]:
shuffle(dataset)

In [87]:
dataset[2]

array(['what is the use of the most productive nuclear power station if a minor fault is enough to make not only the immediate surroundings but the whole region uninhabitable for decades',
       'cual es el uso de la central nuclear mas productiva si un minimo fallo basta para que no solamente sus alrededores mas inmediatos sino tambien toda la region resulten inhabitables durante decadas'],
      dtype='<U4092')

In [107]:
unique = []
words = []
for pair in dataset:
#     print(pair)
    for sentence in pair:
#         print(sentence)
        for word in sentence.split(' '):
#             print(word)
            words.append(word)
            if word not in unique:
                unique.append(word)
#     break  
    
print(f'a number of unique words - {len(unique)}')
print(f'a total number of words - {len(words)}')  

a number of unique words - 39211
a total number of words - 1021109


In [35]:
train, test = dataset[:16000], dataset[16000:]

In [37]:
len(train)

16000

In [38]:
joblib.dump(dataset, 'dataset.pkl') 

['dataset.pkl']

In [39]:
joblib.dump(train, 'train.pkl')
joblib.dump(test, 'test.pkl') 

['test.pkl']

In [40]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

/Users/sonik/anaconda3/envs/pythondata/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
